# Linear Programming — Exam I Review Notes

This notebook covers the core concepts from the LP review, including the graphical method, tableau forms, pivot operations, and the Simplex Algorithm.

## 1. Overview

Given an objective function $z$ and $m$ constraints, we want to find the **maximum** and **minimum** of the objective function within the **feasible region**.

### 1.1 Feasibility
A linear program is **feasible** if the constraints allow for the objective function to be extremified.

- **Graphically**: Constraint gradients define the region in which the objective can be extremified. If constraints *conflict* (i.e., the feasible region cannot form a bounded region), the LP is **infeasible**.
- **Algebraically**: A tableaux with a negative RHS and an entirely negative column indicates infeasibility.

### 1.2 Boundedness
A linear program is **unbounded** when the objective function grows/shrinks toward the extrema without bound.

- **Graphically**: Visible when the feasible region is open in the direction of optimization.
- **Algebraically**: A negative RHS with an entirely negative column in the tableaux signals unboundedness.

## 2. The Graphical Method

The graphical method solves LPs in 2D using constraint gradients and the objective gradient to identify the extrema.

**Steps:**

1. **Compute** the gradient $\nabla g_i$ of each constraint function.
2. **Sketch** the boundary line $g_i \pm s_i = b_i$. The boundary slope is perpendicular to $\nabla g_i$.
3. **Draw arrows** in the direction of feasibility:
   - $\geq$: feasibility is in the direction of $\nabla g_i$
   - $\leq$: feasibility is opposite to $\nabla g_i$
4. **Determine** the feasibility vector $v_i$ where $v_i = (−g_{i,x_2},\; g_{i,x_1})$ and dot it with $\nabla g_i$.
5. **Take** $\nabla z$ and dot it with the result of $v_i \cdot \nabla g_i$.
6. **Use** the result to determine which direction $z$ increases fastest — this is parallel to $v_i$ and runs along $b_i$ toward the extrema.
7. **Check** if the solution is a level set. If so, form the convex hull of the endpoints $w^*, x^*$ for the argmin/argmax.
8. **Plug** the argmax/argmin into the objective function.

### Key Definitions
- **Level Set**: The entire line (or portion thereof) is an argextrema.
- **Convex Hull**: Given two endpoints of a level set, the convex hull is $\{\alpha w^* + (1-\alpha) x^* \mid 0 \leq \alpha \leq 1\}$.

### Example 1 — Find min & max of $f(x_1, x_2) = 6x_1 + x_2 - 2$

$$z = 6x_1 + x_2 - 2$$

Subject to:
$$3x_1 - 2x_2 \leq 12$$
$$2x_1 + 5x_2 \geq -20$$
$$-x_1 - 6x_2 \geq -12$$
$$-x_1 + 2x_2 \leq 4$$
$$6x_1 + x_2 \geq -30$$

| $\nabla g_i$ | $v_i$ | $\nabla z$ | Sign |
|---|---|---|---|
| $(3,-2)$ | $(2,3)$ | $(6,1)$ | $>0$ |
| $(2,5)$ | $(-5,2)$ | $(6,1)$ | $<0$ |
| $(-1,-6)$ | $(6,-1)$ | $(6,1)$ | $>0$ |
| $(-1,2)$ | $(-2,-1)$ | $(6,1)$ | $<0$ |
| $(6,1)$ | $(-1,6)$ | $(6,1)$ | $=0$ ← **Level Set** |

**Intersection (maximum point):** $3x_1 - 2x_2 = 12$ and $-x_1 - 6x_2 = -12$

$$-20x_2 = -24 \implies x_2 = \frac{6}{5}, \quad x_1 = \frac{24}{5}$$

$$\text{argmax} = \left(\frac{24}{5}, \frac{6}{5}\right), \quad \max z = 6\cdot\frac{24}{5} + \frac{6}{5} - 2 = 28$$

**Intersection (minimum level set endpoints):**

$w^* = \left(-\frac{65}{13}, -\frac{49}{13}\right)$, $\quad x^* = \left(-\frac{64}{13}, -\frac{6}{13}\right)$

$$\text{argmin} = \{\alpha w^* + (1-\alpha)x^* \mid 0 \leq \alpha \leq 1\}, \quad \min z = g_5 - 2 = -32$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

# Example 1: Graphical verification
# max z = 6x1 + x2 - 2
# Constraints (all rewritten as <= for linprog, which minimizes)

# Using scipy.optimize.linprog to verify (minimizes c @ x)
c = [-6, -1]  # negate for maximization

# Inequality constraints A_ub @ x <= b_ub
A_ub = [
    [ 3, -2],   # 3x1 - 2x2 <= 12
    [-2, -5],   # 2x1 + 5x2 >= -20  =>  -2x1 - 5x2 <= 20
    [ 1,  6],   # -x1 - 6x2 >= -12  =>  x1 + 6x2 <= 12
    [-1,  2],   # -x1 + 2x2 <= 4
    [-6, -1],   # 6x1 + x2 >= -30   =>  -6x1 - x2 <= 30
]
b_ub = [12, 20, 12, 4, 30]

res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=[(None, None), (None, None)], method='highs')
x1_max, x2_max = res.x
max_z = 6*x1_max + x2_max - 2

print(f"Max argmax: x1 = {x1_max:.4f}, x2 = {x2_max:.4f}")
print(f"Max z = {max_z:.4f}")
print(f"Expected: x1 = {24/5:.4f}, x2 = {6/5:.4f}, z = 28")

Max argmax: x1 = 4.8000, x2 = 1.2000
Max z = 28.0000
Expected: x1 = 4.8000, x2 = 1.2000, z = 28


### Example 2 — Find max of $z = 5x + 4y$

Subject to:
$$6x + 4y \leq 24$$
$$x + 2y \leq 6$$
$$x + y \geq 5 \;\text{and}\; -x - y \geq -5$$

| $\nabla g_i$ | $v_i$ | $\nabla z$ | Sign |
|---|---|---|---|
| $(6,4)$ | $(-4,6)$ | $(5,4)$ | $>0$ |
| $(1,2)$ | $(-2,1)$ | $(5,4)$ | $<0$ |
| $(1,1)$ | $(-1,1)$ | $(5,4)$ | $<0$ |
| $(-1,-1)$ | $(1,-1)$ | $(5,4)$ | $>0$ |

**Intersection:** $6x + 4y = 24$ and $x + y = 5$ gives $2x = 4 \implies x=2,\; y=3$.

$$\max z = 5(2) + 4(3) = 22$$

In [ ]:
# Example 2 verification
c2 = [-5, -4]
A_ub2 = [
    [6, 4],
    [1, 2],
    [-1, -1],  # x + y >= 5
]
b_ub2 = [24, 6, -5]

res2 = linprog(c2, A_ub=A_ub2, b_ub=b_ub2, bounds=[(None,None),(None,None)], method='highs')
x_max, y_max = res2.x
print(f"Max argmax: x = {x_max:.4f}, y = {y_max:.4f}")
print(f"Max z = {5*x_max + 4*y_max:.4f}  (expected: 22)")

AttributeError: y

## 3. Matrix Form

A linear program can be expressed in matrix form using four components:

| Component | Description |
|---|---|
| $A$ | Matrix of coefficients on constraint functions |
| $b$ | Column vector of boundary values |
| $c$ | Row vector of objective function coefficients |
| $d$ | Constant term in the objective function |

Standard form: $\min\; c^\top x + d$ subject to $Ax = b,\; x \geq 0$.

**Note**: Non-negativity ($x \geq 0$) is guaranteed through **free variable substitution**: $x_i = u_i - u_{i'}$ where $u_i, u_{i'} \geq 0$.

### Free Variable Substitution
For unrestricted variable $x_1$, substitute $x_1 = u_1 - u_2$ (both $\geq 0$).
This doubles the variable count but enforces non-negativity.

### Quiz 2 Example — Define $c, d, x, A, b$

$$\max\; z = -6x_1 + 9x_2 - 8$$

Subject to:
$$3x_1 - 5x_2 \leq 15$$
$$-2x_1 + 3x_2 \geq -14$$
$$5x_1 + 4x_2 \geq 20$$
$$5x_1 + x_2 \leq 20$$

**Free variable substitution:** $x_1 = u_1 - u_2$, $x_2 = u_3 - u_4$

So $z = -6(u_1 - u_2) + 9(u_3 - u_4) - 8 = -6u_1 + 6u_2 + 9u_3 - 9u_4 - 8$

**Equivalently minimizing:** $\min\; 6u_1 - 6u_2 - 9u_3 + 9u_4 + 8$

$$c = (-6,\; 6,\; 9,\; 0,\; 0,\; 0,\; 0), \quad d = -8$$

**Adding slack variables** to convert inequalities to equalities:

| Constraint | Standard Form |
|---|---|
| $3x_1 - 5x_2 \leq 15$ | $g_1 + s_1 = 15$ |
| $-2x_1 + 3x_2 \geq -14$ | $g_2 - s_2 = -14$ |
| $5x_1 + 4x_2 \geq 20$ | $g_3 - s_3 = 20$ |
| $5x_1 + x_2 \leq 20$ | $g_4 + s_4 = 20$ |

$$A = \begin{pmatrix} -3 & 3 & 5 & -5 & 1 & 0 & 0 & 0 \\ -2 & 2 & 7 & -7 & 0 & -1 & 0 & 0 \\ 5 & -5 & 4 & -4 & 0 & 0 & -1 & 0 \\ -5 & 5 & -4 & 4 & 0 & 0 & 0 & 1 \end{pmatrix}, \quad b = \begin{pmatrix} 15 \\ -14 \\ 20 \\ 20 \end{pmatrix}$$

## 4. Extended Tableau Form

The **Extended Tableau** displays basic and non-basic variables alongside the objective function so we can perform elementary row operations to find an extrema.

**Structure:**

| $u_1$ | $u_2$ | $\cdots$ | $u_n$ | $s_1$ | $\cdots$ | $s_m$ | $-z$ | RHS |
|---|---|---|---|---|---|---|---|---|
| $\vdots$ | | | | | | | | $b_i$ |
| $c_1$ | $c_2$ | | $c_n$ | $0$ | | $0$ | $1$ | $d$ |

### Basic vs. Non-Basic Variables
- **Basic variables** ($u_1, \ldots, u_n$): the *control space* — free variable substitutions forming the grid in which constraints lie.
- **Non-basic variables** ($s_1, \ldots, s_m$): represent $z = S_i$, the value of boundary hyperplanes within the control space.

**Key rule**: The tableau represents the system with non-basic variables set to zero — the current vertex of the feasible region.

### Example — Extended Tableau for $\max\; z = -6x_1 + 9x_2 - 8$

With free variable sub and slack variables:

$$\begin{pmatrix} u_1 & u_2 & u_3 & u_4 & s_1 & s_2 & s_3 & s_4 & -z & \text{RHS} \\
3 & -3 & -5 & 5 & 1 & 0 & 0 & 0 & 0 & 15 \\
2 & -2 & -7 & 7 & 0 & 1 & 0 & 0 & 0 & 14 \\
-5 & 5 & -4 & 4 & 0 & 0 & 1 & 0 & 0 & -20 \\
5 & -5 & 4 & -4 & 0 & 0 & 0 & 1 & 0 & 20 \\
-6 & 6 & 9 & -9 & 0 & 0 & 0 & 0 & 1 & 8
\end{pmatrix}$$

## 5. The Pivot Operation

A **pivot** alters the control space by swapping basic variable $u_i$ with non-basic variable $s_j$. This moves us to an adjacent vertex of the feasible region.

**Steps to pivot on entry $a_{ij}$ (row $i$, column $j$):**

1. **Locate** the pivot element $a_{ij}$.
2. **Divide** row $i$ by $a_{ij}$ (pivot row becomes $\text{row}_i / a_{ij}$).
3. **Eliminate** column $j$ in all other rows using elementary row operations:
   $$\text{row}_k^{\text{new}} = \text{row}_k^{\text{old}} - a_{kj} \cdot \text{row}_i^{\text{new}} \quad \forall k \neq i$$
4. **Swap** column labels: $u_i \leftrightarrow s_j$.

## 6. Concise Row Form

**Concise Row Form** is a truncated tableau representation: $g_i + b_i = s_i$.

### Pivot Formula in Concise Row Form

Pivoting on entry $a_{ij}$ transforms entries according to:

$$\begin{pmatrix} p & q \\ r & s \end{pmatrix} \longrightarrow \begin{pmatrix} 1/p & -q/p \\ r/p & s - qr/p \end{pmatrix}$$

More precisely:
- Entry $a_{ij}$ → $\dfrac{1}{a_{ij}}$
- Entry $a_{ik}$ (same row, $k \neq j$) → $\dfrac{a_{ik}}{a_{ij}}$
- Entry $a_{kj}$ (same column, $k \neq i$) → $\dfrac{-a_{kj}}{a_{ij}}$
- Entry $a_{k_1 k_2}$ (elsewhere) → $a_{k_1 k_2} - \dfrac{a_{k_1 j} \cdot a_{i k_2}}{a_{ij}}$
- RHS $b_i$ → $\dfrac{b_i}{a_{ij}}$
- RHS $b_k$ ($k \neq i$) → $b_k - \dfrac{a_{kj} \cdot b_i}{a_{ij}}$

In [5]:
import numpy as np

def pivot_extended(T, i, j):
    """
    Perform a pivot on extended tableau T at row i, column j.
    Returns the new tableau.
    """
    T = T.astype(float).copy()
    pivot = T[i, j]
    assert pivot != 0, "Pivot element cannot be zero"
    
    T[i] = T[i] / pivot          # Step 1: divide pivot row by pivot element
    for k in range(len(T)):       # Step 2: eliminate column j in all other rows
        if k != i:
            T[k] = T[k] - T[k, j] * T[i]
    return T


def pivot_concise(T, i, j):
    """
    Perform a pivot on concise row-form tableau T at row i, column j.
    T has shape (m, n) where last column is RHS (b).
    """
    T = T.astype(float).copy()
    p = T[i, j]
    assert p != 0, "Pivot element cannot be zero"
    
    new_T = np.zeros_like(T)
    
    for r in range(T.shape[0]):
        for c in range(T.shape[1]):
            if r == i and c == j:
                new_T[r, c] = 1.0 / p
            elif r == i:
                new_T[r, c] = T[r, c] / p
            elif c == j:
                new_T[r, c] = -T[r, c] / p
            else:
                new_T[r, c] = T[r, c] - (T[r, j] * T[i, c]) / p
    return new_T


print("Pivot helpers defined.")
print("Usage: pivot_extended(T, row_index, col_index)")
print("Usage: pivot_concise(T, row_index, col_index)")

Pivot helpers defined.
Usage: pivot_extended(T, row_index, col_index)
Usage: pivot_concise(T, row_index, col_index)


## 7. The Simplex Algorithm

The Simplex Algorithm iteratively pivots to reach an optimal tableau.

### Setup
Put the LP into **concise tableau form**.

---

### Phase I — Find a Feasible Basic Solution

Goal: Make all $b_i \geq 0$.

1. Find row $i_0$ where $b_i < 0$.
2. If all $b_i \geq 0$ → move to Phase II.
3. Otherwise, find column $j^*$ where $a_{i_0, j^*} > 0$.
   - No positive entries → **LP is infeasible**.
   - Multiple positive entries → use **Bland's Rule** to break ties.
4. Find a negative entry *above* $a_{i_0, j^*}$ to select pivot row $i^*$:
   - Multiple candidates → use the **ratio test**: pick $i^*$ minimizing $|b_i / a_{i,j^*}|$.
   - No negative entries → $i^* = i_0$.
5. Pivot on $(i^*, j^*)$. Repeat until all $b_i \geq 0$ or LP is infeasible.

---

### Phase II — Optimize

Goal: Make all $c_j \geq 0$ (for minimization).

1. Find a negative entry in the $c$ (cost) vector.
   - Multiple negatives → use **Bland's Rule** to select $j^*$.
   - No negatives → **tableau is optimal** ✓
2. Look for negative entries in column $j^*$ (above the $c$ row).
   - No negative entries → **LP is unbounded**.
   - Multiple candidates → use **ratio test** to pick $i^*$.
   - Ratio test tie → use **Bland's Rule**.
3. Pivot on $(i^*, j^*)$. Repeat until optimal or unbounded.

---

### Bland's Rule

When the ratio test ties (or to prevent cycling), select pivot columns/rows by their **original index order**:
$$u_1, u_2, \ldots, u_n, s_1, s_2, \ldots, s_m$$
This ordering is fixed and never changes.

### Level Set Detection
After reaching an optimal tableau, if there are **non-zero entries above a $c_j = 0$ entry**, a **level set** (multiple optimal solutions) exists. Perform an additional pivot to find the other endpoint.

In [ ]:
import numpy as np

def simplex(c_tableau, b, A, col_names=None, verbose=True):
    """
    Two-phase Simplex Algorithm using the extended tableau.

    Parameters:
    -----------
    c_tableau : 1D array of objective coefficients (for min; negate for max)
    b         : 1D array of RHS values
    A         : 2D array of constraint coefficients (already with slack cols if desired)
    col_names : list of column names (optional, for display)
    
    Builds an extended tableau: [A | I | b] with c row appended.
    Returns the final tableau.
    """
    m, n = A.shape
    # Build full tableau: [A | slacks | b]
    T = np.zeros((m + 1, n + m + 1))
    T[:m, :n] = A
    T[:m, n:n+m] = np.eye(m)
    T[:m, -1] = b
    T[m, :n] = c_tableau
    T[m, -1] = 0  # initial z = 0

    if col_names is None:
        col_names = [f'u{i+1}' for i in range(n)] + [f's{i+1}' for i in range(m)] + ['RHS']
    row_names = [f's{i+1}' for i in range(m)] + ['z']

    def print_tableau():
        header = '  '.join(f'{c:>8}' for c in col_names)
        print(f"{'':>6}  {header}")
        for i, row in enumerate(T):
            row_str = '  '.join(f'{v:>8.3f}' for v in row)
            print(f"{row_names[i]:>6}  {row_str}")
        print()

    if verbose:
        print("=== Initial Tableau ===")
        print_tableau()

    # Phase I: make all b >= 0
    phase = 1
    iteration = 0
    while True:
        iteration += 1
        neg_b = [i for i in range(m) if T[i, -1] < -1e-9]
        if not neg_b:
            if verbose: print("=== Phase I complete: all b_i >= 0 ===")
            break
        
        i0 = neg_b[0]  # first negative b
        pos_cols = [j for j in range(T.shape[1]-1) if T[i0, j] > 1e-9]
        if not pos_cols:
            print("LP is INFEASIBLE")
            return T
        j_star = pos_cols[0]  # Bland's rule: pick smallest index
        
        # Find pivot row: negative entry in col j_star above row m
        neg_rows = [i for i in range(m) if T[i, j_star] < -1e-9]
        if not neg_rows:
            i_star = i0
        else:
            # Ratio test
            i_star = min(neg_rows, key=lambda i: abs(T[i, -1] / T[i, j_star]))
        
        if verbose: print(f"Phase I pivot: row={row_names[i_star]}, col={col_names[j_star]}")
        T = pivot_extended(T, i_star, j_star)
        row_names[i_star] = col_names[j_star]
        if verbose: print_tableau()

    # Phase II: optimize
    while True:
        neg_c = [j for j in range(T.shape[1]-1) if T[m, j] < -1e-9]
        if not neg_c:
            if verbose: print("=== Optimal tableau reached ===")
            break
        j_star = neg_c[0]  # Bland's rule
        
        neg_rows = [i for i in range(m) if T[i, j_star] < -1e-9]
        if not neg_rows:
            print("LP is UNBOUNDED")
            return T
        
        # Ratio test: minimize |b_i / a_ij|
        i_star = min(neg_rows, key=lambda i: abs(T[i, -1] / T[i, j_star]))
        
        if verbose: print(f"Phase II pivot: row={row_names[i_star]}, col={col_names[j_star]}")
        T = pivot_extended(T, i_star, j_star)
        row_names[i_star] = col_names[j_star]
        if verbose: print_tableau()

    print(f"Optimal z = {T[m, -1]:.4f}")
    print("Basic variable values:")
    for i in range(m):
        print(f"  {row_names[i]} = {T[i, -1]:.4f}")
    return T


print("Simplex function defined. Run the cell below for a worked example.")

### HW4 Example — $\min\; z = 6y + 2z + 6$

Subject to:
$$3y - 5z \geq -4$$
$$-2y + 7z \geq 20$$
$$y + 2z \leq 3$$
$$-5y - 2z \leq 8$$

In [ ]:
# HW4 Example: min z = 6y + 2z_var + 6
# Converted to standard form (Ax <= b where needed, negated for >=):
# 3y - 5z >= -4   =>  -3y + 5z <= 4
# -2y + 7z >= 20  =>  2y - 7z <= -20
# y + 2z <= 3
# -5y - 2z <= 8

# Using scipy linprog (minimizes c @ x, all constraints A_ub @ x <= b_ub)
c_obj = [6, 2]  # minimize 6y + 2z + 6 (constant 6 doesn't affect argmin)

A_hw = [
    [-3,  5],   # -3y + 5z <= 4
    [ 2, -7],   # 2y - 7z <= -20
    [ 1,  2],   # y + 2z <= 3
    [-5, -2],   # -5y - 2z <= 8
]
b_hw = [4, -20, 3, 8]

res_hw = linprog(c_obj, A_ub=A_hw, b_ub=b_hw,
                 bounds=[(None, None), (None, None)], method='highs')

if res_hw.success:
    y_opt, z_opt = res_hw.x
    min_z = 6*y_opt + 2*z_opt + 6
    print(f"Optimal: y = {y_opt:.4f}, z = {z_opt:.4f}")
    print(f"Min z = {min_z:.4f}")
else:
    print(f"Solver status: {res_hw.message}")

### HW5 Example — $\max\; z = 2y + 2z + 2$

Subject to:
$$3y + 8z \leq 48$$
$$4y + 2z \leq 40$$
$$2y + 3z \leq 24$$

In [ ]:
# HW5: max z = 2y + 2z_var + 2
# min -z = -2y - 2z_var
c_hw5 = [-2, -2]
A_hw5 = [
    [3, 8],
    [4, 2],
    [2, 3],
]
b_hw5 = [48, 40, 24]

res_hw5 = linprog(c_hw5, A_ub=A_hw5, b_ub=b_hw5,
                  bounds=[(0, None), (0, None)], method='highs')

y_opt5, z_opt5 = res_hw5.x
max_z5 = 2*y_opt5 + 2*z_opt5 + 2
print(f"Optimal: y = {y_opt5:.4f}, z = {z_opt5:.4f}")
print(f"Max z = {max_z5:.4f}")

## 8. Summary — Algorithm Decision Tree

```
Set up LP in standard form (Ax = b, x >= 0)
         │
         ▼
    Concise Tableau
         │
    ┌────▼────┐
    │ PHASE I │  ← Any b_i < 0?
    └────┬────┘
         │ All b_i >= 0
    ┌────▼─────┐
    │ PHASE II │  ← Any c_j < 0?
    └────┬─────┘
         │ All c_j >= 0
    ┌────▼──────┐
    │  OPTIMAL  │
    └────┬──────┘
         │ Any non-zero entries above c_j = 0?
    ┌────▼──────────┐
    │  LEVEL SET?   │  ← Additional pivot to find 2nd endpoint
    └───────────────┘
```

| Condition | Conclusion |
|---|---|
| Phase I: no positive $a_{i_0,j}$ | **Infeasible** |
| Phase II: no negative $a_{i,j^*}$ | **Unbounded** |
| All $c_j \geq 0$ | **Optimal** |
| $c_j = 0$ with non-zero column entries | **Level Set** (multiple optima) |

### Bland's Rule — Pivot Selection Order
$$u_1 < u_2 < \cdots < u_n < s_1 < s_2 < \cdots < s_m$$
Always pick the **smallest-indexed** eligible column (Phase II) or row (ratio test tie).

## 9. Quick Reference

| Concept | Key Formula / Rule |
|---|---|
| Feasibility | Tableau has negative RHS + entirely negative column |
| Boundedness | Negative $c_j$ with all-positive column above it |
| Graphical extrema direction | $\nabla z \cdot v_i$ where $v_i = (-g_{i,x_2}, g_{i,x_1})$ |
| Pivot (row $i$, col $j$) | $\text{row}_i / a_{ij}$; then eliminate column |
| Concise pivot: $a_{ij}$ | $\to 1/a_{ij}$ |
| Concise pivot: same row $k\neq j$ | $\to a_{ik}/a_{ij}$ |
| Concise pivot: same col $k\neq i$ | $\to -a_{kj}/a_{ij}$ |
| Concise pivot: elsewhere $(k_1,k_2)$ | $\to a_{k_1k_2} - a_{k_1j}\cdot a_{ik_2}/a_{ij}$ |
| Convex hull (level set) | $\{\alpha w^* + (1-\alpha)x^* \mid 0\leq\alpha\leq 1\}$ |